# Limpieza de Max

## Pedir datos a la API

In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import requests as rq

import sys, os
sys.path.append(os.path.abspath(".."))

from src.buscar_peliculas import buscar_peliculas

Pedimos los datos:

In [2]:
dfmax_raw = buscar_peliculas('Max')

<Response [200]> 5 1182


Fecha de consulta en la API: 21-10-2025 1:11

Verificamos que no haya perdida de datos:

In [3]:
largo_dataframe = len(dfmax_raw[0])
largo_segun_API = dfmax_raw[1]
print([largo_dataframe, largo_segun_API])

[1182, 1182]


Los largos coinciden

In [4]:
dfmax1 = dfmax_raw[0]

In [5]:
dfmax1

,id,title,year,imdb_id,tmdb_id,tmdb_type,type
0,1703330,Superman,2025,tt5950044,1061474,movie,movie
1,1783110,Sinners,2025,tt31193180,1233413,movie,movie
2,1532981,The Batman,2022,tt1877830,414906,movie,movie
3,1718402,Heretic,2024,tt28015403,1138194,movie,movie
4,1653388,Mickey 17,2025,tt12299608,696506,movie,movie
...,...,...,...,...,...,...,...
1177,1307384,Presenting Princess Shaw,2016,tt4161074,357799,movie,movie
1178,1675301,Master of Light,2022,tt17660502,934624,movie,movie
1179,1613151,Stockton on My Mind,2020,tt11905696,712393,movie,movie
1180,540037,Brittany Murphy: An ID Mystery,2020,tt12323826,709098,movie,movie


Guardamos estos datos crudos en un csv

In [6]:
dfmax1.to_csv('../data/raw_api_max.csv', index=False)

## Limpieza
### Columnas
Las columnas nos entregan los siguientes datos:
- `id`: Identificador de la pelicula en la API
- `title`: Nombre de la pelicula
- `year`: Año de estreno de la pelicula
- `imdb_id`: Identificador de la pelicula en IMDB
- `tmdb_id`: Identificador de la pelicula en TMDB
- `tmdb_type`: Tipo del titulo en TMDB
- `type`: Tipo del titulo en la API

### Eliminar Columnas
Como le especificamos a la API que nos entregara solo peliculas, la columna `type` y `tmdb_type` no deberian entregar informacion importante, ya que todas las filas tienen el mismo tipo

In [7]:
dfmax1['type'].unique()

array(['movie'], dtype=object)

In [8]:
dfmax1['tmdb_type'].unique()

array(['movie'], dtype=object)

Al ser todos del mismo tipo, ambas columnas pueden ser eliminadas

In [9]:
dfmax2 = dfmax1[['id', 'title', 'year', 'imdb_id', 'tmdb_id']]

### Valores Na

In [11]:
resultados = {}
for i in dfmax2.columns:
    resultados[i] = dfmax2[i].isna().sum()
resultados

{'id': np.int64(0),
 'title': np.int64(0),
 'year': np.int64(0),
 'imdb_id': np.int64(0),
 'tmdb_id': np.int64(0)}

### Duplicados

In [12]:
dfmax2.duplicated().sum()

np.int64(0)

#### imdb_id

In [13]:
dfmax2.duplicated('imdb_id').sum()

np.int64(0)

#### id
Deberia ser unico para cada pelicula

In [14]:
dfmax2.duplicated('id').sum()

np.int64(0)

#### title
Es posible que se repita, ya que hay peliculas con el mismo titulo

In [15]:
dfmax2.duplicated('title').sum()

np.int64(11)

revisemos porque se repiten los titulos de las peliculas

In [16]:
dfmax2[dfmax2.duplicated('title')].head()

,id,title,year,imdb_id,tmdb_id
270,1260826,Mortal Kombat,1995,tt0113855,9312
278,19750,A Nightmare on Elm Street,1984,tt0087800,377
283,1365622,Superman,1978,tt0078346,1924
375,1318122,Red Dawn,1984,tt0087985,1880
558,1376890,The Amityville Horror,1979,tt0078767,11449


In [18]:
dfmax2[dfmax2['title'] == 'Mortal Kombat']

,id,title,year,imdb_id,tmdb_id
41,1559743,Mortal Kombat,2021,tt0293429,460465
270,1260826,Mortal Kombat,1995,tt0113855,9312


Son distintas peliculas con mismo nombre

### Tipos de columnas

In [19]:
dfmax2.dtypes

id          int64
title      object
year        int64
imdb_id    object
tmdb_id     int64
dtype: object

Los tipos de las columnas estan bien

### Analisis de columna `year`

In [20]:
dfmax2['year'].describe()

count    1182.000000
mean     1992.862098
std        29.776084
min      1921.000000
25%      1966.250000
50%      2006.000000
75%      2019.000000
max      2025.000000
Name: year, dtype: float64

todos estan dentro de un rango logico

## Creacion archivo `.csv`
Como ya tenemos los datos correctos y limpios, procedemos a guardar en un archivo `.csv`

In [21]:
dfmax2.to_csv('../data/api_max.csv', index=False)